# Group Theory in Quantum Compilation

This notebook demonstrates how group theory concepts are used *concretely* in quantum computing, especially for quantum compilation tasks.

We will use PennyLane to illustrate:
- Euler decomposition of single-qubit gates (SU(2))
- Clifford group circuits
- Parametrized templates for unitary synthesis

In [1]:
import pennylane as qml
from pennylane import numpy as np

## 1. Euler Decomposition of SU(2) Unitaries

Any single-qubit unitary is just a global phase (which doesn’t affect measurement probabilities) times an $SU(2)$ rotation on the Bloch sphere.

- $U(2)$: set of all 2x2 unitary matrices, where $U^\dagger U = I$. In general, $\det(U) = e^{i\delta}$, where $\delta$ is a global phase.
- $SU(2)$: $U(2)$ matrices with determinant 1, where $\det(U) = 1$.

Any element of $U(2)$ can be written as a global phase times an element of $SU(2)$ as in $SU(2) \times U(1)$ (global phase separation), where $U(1)$ represents the global phase factor $e^{i\delta}$.

Therefore, any single-qubit unitary in $SU(2)$ can be decomposed as:

$
U = e^{i\delta} RZ(\alpha) \cdot RY(\beta) \cdot RZ(\gamma) = e^{i\delta} e^{i\alpha Z} e^{i\beta Y} e^{i\gamma Z}
$

This decomposition is a direct consequence of the group structure of $SU(2)$.

PennyLane provides tools to compute this decomposition automatically.

In [2]:
# Define an arbitrary single-qubit unitary (composition of rotations)
U = qml.matrix(qml.RX(0.3, wires=0)) @ qml.matrix(qml.RY(0.5, wires=0)) @ qml.matrix(qml.RZ(0.8, wires=0))

# Perform Euler angle decomposition (RZ-RY-RZ)
ops = qml.ops.one_qubit_decomposition(U, wire=0, return_global_phase=True)

# Extract angles and global phase
alpha = ops[0].data[0]  # RZ
beta = ops[1].data[0]   # RY
gamma = ops[2].data[0]  # RZ
global_phase = ops[3].data[0]  # GlobalPhase

print("Euler angles (RZ-RY-RZ decomposition):")
print(f"alpha = {alpha:.4f}")
print(f"beta = {beta:.4f}")
print(f"gamma = {gamma:.4f}")
print(f"global phase = {global_phase:.4f}")


Euler angles (RZ-RY-RZ decomposition):
alpha = 1.3730
beta = 0.5765
gamma = 12.0705
global phase = 0.0000


These angles are not the original parameters 0.3, 0.5, 0.8. They are the Euler angles that reproduce the same matrix in the RZ-RY-RZ convention.

We can reconstruct any single-qubit unitary using the Euler angles $\alpha$, $\beta$, $\gamma$ and the global phase $\delta$.

In [3]:
# Build the gates as matrices
RZ_alpha = qml.matrix(qml.RZ(alpha, wires=0))
RY_beta  = qml.matrix(qml.RY(beta, wires=0))
RZ_gamma = qml.matrix(qml.RZ(gamma, wires=0))

# Combine to reconstruct U
U_reconstructed = RZ_gamma @ RY_beta @ RZ_alpha

print("Reconstructed U matrix:")
print(U_reconstructed)
print("Is the same as the original matrix?", np.allclose(U, U_reconstructed))

Reconstructed U matrix:
[[ 0.86800903-0.40712854j -0.16893051-0.22862449j]
 [ 0.16893051-0.22862449j  0.86800903+0.40712854j]]
Is the same as the original matrix? True


## Clifford Group Circuits

The Clifford group is a discrete subgroup of unitaries generated by:
- Hadamard (H) converts |0⟩ to (|0⟩ + |1⟩)/√2 and |1⟩ to (|0⟩ - |1⟩)/√2.
- Phase (S) adds a phase of $\pi/2$ to the |1⟩ state. |0⟩ remains unchanged.
- CNOT (CNOT) flips the second qubit if the first qubit is |1⟩.

$
H = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}, \quad
S = \begin{pmatrix} 1 & 0 \\ 0 & i \end{pmatrix}, \quad
\text{CNOT} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}
$

In [4]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def clifford_circuit():
    qml.Hadamard(0)
    qml.CNOT(wires=[0, 1])
    qml.S(0)
    return qml.state()

state = clifford_circuit()
print(state)


[0.70710678+0.j         0.        +0.j         0.        +0.j
 0.        +0.70710678j]


This circuit is entirely made of Clifford gates. 
These form a group under composition, with important applications:

- Efficient classical simulation (Gottesman-Knill)
- Error correction codes
- Stabilizer circuits

Clifford gates map Pauli operators to Pauli operators under conjugation.

For any Clifford gate $C$ and any Pauli $P$, we have:

$
C P C^\dagger = P'
$

where $P'$ is also a Pauli operator (possibly with a phase).

This property makes Clifford circuits **classically simulatable** via the stabilizer formalism.

Here are the single-qubit Pauli matrices:

$
X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad
Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad
Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
$

In [5]:
# Pauli matrices
X = qml.matrix(qml.PauliX(0))
Y = qml.matrix(qml.PauliY(0))
Z = qml.matrix(qml.PauliZ(0))

print("X =\n", X)
print()
print("Y =\n", Y)
print()
print("Z =\n", Z)


X =
 [[0 1]
 [1 0]]

Y =
 [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]

Z =
 [[ 1  0]
 [ 0 -1]]



Hadamard gate $H$ is a Clifford:

$
H X H^\dagger = Z
$

$
H Y H^\dagger = -Y
$

$
H Z H^\dagger = X
$


In [6]:
H = qml.matrix(qml.Hadamard(0))

HXH = H @ X @ H.conj().T
HYH = H @ Y @ H.conj().T
HZH = H @ Z @ H.conj().T

print("H X H† =\n", np.round(HXH, 4))
print()
print("H Y H† =\n", np.round(HYH, 4))
print()
print("H Z H† =\n", np.round(HZH, 4))

H X H† =
 [[ 1. -0.]
 [ 0. -1.]]

H Y H† =
 [[0.-0.j 0.+1.j]
 [0.-1.j 0.+0.j]]

H Z H† =
 [[-0.  1.]
 [ 1. -0.]]


The phase gate $S$ is also Clifford:

$
S X S^\dagger = Y
$

$
S Y S^\dagger = -X
$

$
S Z S^\dagger = Z
$

In [7]:
S = qml.matrix(qml.S(0))

SXS = S @ X @ S.conj().T
SYS = S @ Y @ S.conj().T
SZS = S @ Z @ S.conj().T

print("S X S† =\n", np.round(SXS, 4))
print()
print("S Y S† =\n", np.round(SYS, 4))
print()
print("S Z S† =\n", np.round(SZS, 4))

S X S† =
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]

S Y S† =
 [[ 0.+0.j -1.+0.j]
 [-1.+0.j  0.+0.j]]

S Z S† =
 [[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


CNOT is a Clifford gate that preserves the Pauli group under conjugation.

For control=0, target=1:

- $X \otimes I \quad\rightarrow\quad  X \otimes X$
- $I \otimes X \quad\rightarrow\quad I \otimes X$
- $Y \otimes I \quad\rightarrow\quad Y \otimes X$
- $I \otimes Y \quad\rightarrow\quad Z \otimes Y$
- $Z \otimes I \quad\rightarrow\quad Z \otimes I$
- $I \otimes Z \quad\rightarrow\quad Z \otimes Z$

For example, 

$
\text{CNOT} (X \otimes I) = |0⟩⟨0| \otimes X + |1⟩⟨1| \otimes X
$

CNOT is unitary and Hermitian, so $\text{CNOT}^\dagger = \text{CNOT}^T$.

$
\text{CNOT} (I \otimes X) \text{CNOT}^\dagger = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix} 
\begin{pmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}
\begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix} = \begin{pmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix} = I \otimes X
$

In [8]:
# Pauli matrices
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Kronecker (tensor) products for 2 qubits
XI = np.kron(X, I)
IX = np.kron(I, X)
YI = np.kron(Y, I)
IY = np.kron(I, Y)
ZI = np.kron(Z, I)
IZ = np.kron(I, Z)

# CNOT matrix (control=0, target=1)
CNOT = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,0,1],
    [0,0,1,0]
])

def conjugate(op, gate):
    return gate @ op @ gate.conj().T

# Conjugations
print("CNOT (X⊗I) CNOT† =\n", np.round(conjugate(XI, CNOT), 4))
print()
print("CNOT (I⊗X) CNOT† =\n", np.round(conjugate(IX, CNOT), 4))
print()
print("CNOT (Y⊗I) CNOT† =\n", np.round(conjugate(YI, CNOT), 4))
print()
print("CNOT (I⊗Y) CNOT† =\n", np.round(conjugate(IY, CNOT), 4))
print()
print("CNOT (Z⊗I) CNOT† =\n", np.round(conjugate(ZI, CNOT), 4))
print()
print("CNOT (I⊗Z) CNOT† =\n", np.round(conjugate(IZ, CNOT), 4))


CNOT (X⊗I) CNOT† =
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

CNOT (I⊗X) CNOT† =
 [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

CNOT (Y⊗I) CNOT† =
 [[0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]

CNOT (I⊗Y) CNOT† =
 [[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

CNOT (Z⊗I) CNOT† =
 [[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

CNOT (I⊗Z) CNOT† =
 [[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]


We see:

- $\text{CNOT} (X \otimes I) \text{CNOT}^\dagger = X \otimes X$
- $\text{CNOT} (I \otimes X) \text{CNOT}^\dagger = I \otimes X$
- $\text{CNOT} (Y \otimes I) \text{CNOT}^\dagger = Y \otimes X$
- $\text{CNOT} (I \otimes Y) \text{CNOT}^\dagger = Z \otimes Y$
- $\text{CNOT} (Z \otimes I) \text{CNOT}^\dagger = Z \otimes I$
- $\text{CNOT} (I \otimes Z) \text{CNOT}^\dagger = Z \otimes Z$

Therefore, the CNOT gate preserves the Pauli group under conjugation.

### Why This Matters

- Clifford gates transform Pauli operators into Pauli operators under conjugation.
- Clifford gates transform stabilizer states into stabilizer states.
- Stabilizer states are defined as the +1 eigenstates of a set of commuting Pauli operators (the **stabilizers**).

Suppose $S$ is a stabilizer for a state $|\psi\rangle$:

$
S|\psi\rangle = |\psi\rangle
$

and $C$ is a Clifford gate, then:

$
(C S C^\dagger) (C|\psi\rangle) = C |\psi\rangle 
$


Therefore, instead of tracking full quantum state vectors (which grow exponentially), 
we only track **the stabilizers** (which are just Pauli strings, polynomial in number).  

This is **efficient on a classical computer**.

### Example 1: Hadamard on |0⟩

- Initial state: $|0\rangle$
- Stabilizer:

    $
    Z|0\rangle = +|0\rangle
    $

- Apply Hadamard:

    $
    H Z H^\dagger = X
    $

- New stabilizer: **X**

    $
    H|0\rangle = |+\rangle
    $

- Check:

    $
    X|+\rangle = +|+\rangle
    $

**Result:**  
- The state changes from $|0\rangle$ to $|+\rangle$.  
- The stabilizer changes from **Z** to **X**.  
- We only needed to track the Pauli label!

### Example 2: S Gate on |+⟩

- Initial state: 

    $
    |+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}
    $

- Stabilizer:

    $
    X|+\rangle = +|+\rangle
    $

- Apply S gate:

    $
    S X S^\dagger = Y
    $

- New stabilizer: **Y**

**Result:**

- Resulting state:

    $
    S|+\rangle = \frac{|0\rangle + i|1\rangle}{\sqrt{2}}
    $

- Check:

    $
    Y \left(\frac{|0\rangle + i|1\rangle}{\sqrt{2}}\right) = +\left(\frac{|0\rangle + i|1\rangle}{\sqrt{2}}\right)
    $

Take-home message:

- Stabilizer circuits (Clifford-only circuits) are easy to simulate classically.
- Their evolution can be tracked using stabilizer generators (Pauli operators).
- Clifford gates map stabilizers to stabilizers, which can be computed efficiently.

## Parametric Circuit for Approximate Synthesis

We can use a parametric circuit as a template to approximate any SU(2) unitary.

This is the basis for variational quantum compiling.

In [9]:
dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def su2_template(weights):
    qml.RZ(weights[0], wires=0)
    qml.RY(weights[1], wires=0)
    qml.RZ(weights[2], wires=0)
    return qml.expval(qml.PauliZ(0))

# Example usage
weights = np.array([0.1, 0.2, 0.3])
result = su2_template(weights)
print(f"Expectation value: {result:.4f}")

Expectation value: 0.9801


This parametric template uses the Euler decomposition structure to approximate any single-qubit unitary.

By training the parameters, we can "synthesize" arbitrary target unitaries.

This is a core task in:
- Variational compiling
- Gate decomposition with error correction constraints

## Conclusion

We explored how **group theory** appears concretely in quantum computing:

- Euler decomposition of SU(2) → single-qubit gate compilation  
- Clifford group circuits → efficient simulation & error correction  
- Parametric templates → variational synthesis of unitaries  

These techniques form the backbone of many quantum compiler toolchains.


## References

- PennyLane documentation: https://pennylane.ai/
- Nielsen & Chuang, *Quantum Computation and Quantum Information*
- Gottesman-Knill Theorem